In [1]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import time

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

brt = boto3.client(service_name='bedrock-runtime')

def decoder(data):
    decodedBytes = base64.b64decode(data)
    decodedStr = decodedBytes.decode("ascii") 
    json_str=json.loads(decodedStr)
    return json_str

##Processing data to fetch role and content
def data_preprocessing(data):
    #data = json_data['Records'][0]["body"]
    convo = ""
    content = json.loads(data["body"])["body"]["transcript"][0]["Content"]
    role = json.loads(data["body"])["body"]["transcript"][0]["ParticipantRole"]
    convo = convo + role + " : " + content
    #parsed_data = json.loads(data)
    #content = data#["body"]#["transcript"]#[0]#["Content"]
    #role = data["body"]["transcript"][0]["ParticipantRole"]
    
    #convo = ""
    #for i in range(len(transcription['transcriptions'])):
        #convo = convo + transcription['transcriptions'][i]['ParticipantRole'] + ": " + transcription['transcriptions'][i]['Content']
        #convo += "\n"
    #role = transcription["Segments"][0]["Transcript"]["ParticipantRole"]
    #content = transcription["Segments"][0]["Transcript"]["Content"]
    #convo = convo + role + " : " + content
    
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]
    print(f'end_indices:{end_char_indices}')
    end_index = end_char_indices[len(end_char_indices)-1]
    print(f'end_index:{end_index}')
    if end_index == len(data1)-1:
        result = data1[start_index_final:]
    else:
        result = data1[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result
    
#Defining function to connect to Bedrock LLM
def load_claude2(bedrock_runtime , prompt , temp , top_p,top_k):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "top_k":top_k,
            "max_tokens_to_sample": 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-v2", body=json.dumps(body), accept="application/json", contentType="application/json"
                 )
        
        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise

#Defining LLM function for the prompt generator for entity extraction  
entities = "name of patient, status of insurance, insurance number, demographic details etc."
def enrollment_prompt_generator(conversation,entities):
    prompt_claude = """Human: \" """ + conversation + """ \"
 
The above conversation is an automated transcript between a call centre agent and an insurance subscriber or 
patiet. I want to extract few key entities like \" """ + entities + """ \"". All or some information may be present in this transcript.
Extract the entities that you are able to find from this piece of call transcript.
 
The output would be a structured json with only the extracted fields. Just print the exact output without any extra sentences at the end or beggining. 
No need to print any extra text. Also do not generate an answer if that is not found in the transcript.
 
Assistant:
"""
    return prompt_claude

def SQS_Publisher(json_data):
    try:
        sqs = boto3.client('sqs',region_name='us-east-1')
        # Define the SQS queue URL
        queue_url = 'https://sqs.us-east-1.amazonaws.com/383299343633/ch-agent-assist-sqs'
        response = sqs.send_message(QueueUrl= queue_url,MessageBody=json.dumps(json_data) )
    except Exception as e:
        print(e)
        
def SQS_data_postprocessing(event,json_data):
    json_response = {
            "stream": "FORM_DATA",
            "streamConnectionId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
             "body": {
                 "transactionId": "f830e890-3ff2-4fdc-a08e-dd9b78a2dc28",
                  "contactId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
            "form_data": json_data,
                     }  
            }
    return json_response        
    
# Lambda handler to intgerate with AWS
def lambda_handler(event,context):
    start_time = time.time()
    final_transcript = ""
    for i in range(len(event["Records"])):
        final_transcript += "\n" + data_preprocessing(event["Records"][i])
        
    end_time1 = time.time()    # Record the end time

    # Calculate the elapsed time and log it
    elapsed_time1 = end_time1 - start_time
    print(f"Elapsed Time after data_preprocessing: {elapsed_time1} seconds")
    
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities)
    end_time2 = time.time()
    elapsed_time2 = end_time2 - end_time1
    print(f"Elapsed Time after prompt generation: {elapsed_time2} seconds")
    
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    end_time3 = time.time()
    elapsed_time3 = end_time3 - end_time2
    print(f"Elapsed Time after model calling: {elapsed_time3} seconds")
    
    json_data = data_postprocessing(enrollment_data)
    end_time4 = time.time()
    elapsed_time4 = end_time4 - end_time3
    print(f"Elapsed Time after model output postprocessing: {elapsed_time4} seconds")
    
    enrollment_json_object = json.loads(json_data)
    end_time5 = time.time()
    elapsed_time5= end_time5 - end_time4
    print(f"Elapsed Time after converting output data to json: {elapsed_time5} seconds")
    
    #print(event["Records"][0]['body'])
    #print(json.loads(event["Records"][0]['body'])["streamConnectionId"])
    sqs_data = SQS_data_postprocessing(event,enrollment_json_object)
    end_time6 = time.time()
    elapsed_time6 = end_time6 - end_time5
    print(f"Elapsed Time after sqs data postprocessing: {elapsed_time6} seconds")
    
    SQS_Publisher(sqs_data)
    end_time7 = time.time()
    elapsed_time7 = end_time7 - end_time6
    print(f"Elapsed Time after sending data to SQS: {elapsed_time7} seconds")
    return {"statusCode": 200,"body": "Data Sent to SQS Sucessfully"}

In [2]:
data = {
      "Records": [
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"CUSTOMER\",\"ParticipantRole\":\"CUSTOMER\",\"Content\":\"Sure,my insurance is with Anthem INC.\",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    },
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"AGENT\",\"ParticipantRole\":\"AGENT\",\"Content\":\"Thank you,can you share your policy number? \",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    }
  ]
}

# Try 1

In [3]:
lambda_handler(data,'') #26th Feb

Elapsed Time after data_preprocessing: 4.029273986816406e-05 seconds
Elapsed Time after prompt generation: 8.96453857421875e-05 seconds
Elapsed Time after model calling: 13.805869340896606 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.00023055076599121094 seconds
Elapsed Time after converting output data to json: 1.5974044799804688e-05 seconds
Elapsed Time after sqs data postprocessing: 2.9087066650390625e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.06397867202758789 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}

# Try 2 

In [4]:
lambda_handler(data,'') #10.30 --> 28th Feb

Elapsed Time after data_preprocessing: 4.291534423828125e-05 seconds
Elapsed Time after prompt generation: 9.870529174804688e-05 seconds
Elapsed Time after model calling: 1.0602726936340332 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.00014829635620117188 seconds
Elapsed Time after converting output data to json: 1.7881393432617188e-05 seconds
Elapsed Time after sqs data postprocessing: 2.9802322387695312e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.07726097106933594 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}

# Try 3

In [5]:
lambda_handler(data,'')

Elapsed Time after data_preprocessing: 4.0531158447265625e-05 seconds
Elapsed Time after prompt generation: 5.91278076171875e-05 seconds
Elapsed Time after model calling: 0.811582088470459 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.00012373924255371094 seconds
Elapsed Time after converting output data to json: 1.5735626220703125e-05 seconds
Elapsed Time after sqs data postprocessing: 3.743171691894531e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.06293654441833496 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}

# Try 4

In [6]:
lambda_handler(data,'')

Elapsed Time after data_preprocessing: 4.792213439941406e-05 seconds
Elapsed Time after prompt generation: 7.700920104980469e-05 seconds
Elapsed Time after model calling: 3.4205403327941895 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.00012087821960449219 seconds
Elapsed Time after converting output data to json: 1.8358230590820312e-05 seconds
Elapsed Time after sqs data postprocessing: 2.9325485229492188e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.1222991943359375 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}

# Try 5

In [7]:
lambda_handler(data,'')

Elapsed Time after data_preprocessing: 4.291534423828125e-05 seconds
Elapsed Time after prompt generation: 5.745887756347656e-05 seconds
Elapsed Time after model calling: 6.52532696723938 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.000133514404296875 seconds
Elapsed Time after converting output data to json: 2.0265579223632812e-05 seconds
Elapsed Time after sqs data postprocessing: 3.170967102050781e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.055951595306396484 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}

In [8]:
lambda_handler(data,'')

Elapsed Time after data_preprocessing: 4.1961669921875e-05 seconds
Elapsed Time after prompt generation: 5.936622619628906e-05 seconds
Elapsed Time after model calling: 1.9055275917053223 seconds
start_index:1
end_indices:[41]
end_index:41
result:{
  "insurance_provider": "Anthem INC."
}
Elapsed Time after model output postprocessing: 0.00011992454528808594 seconds
Elapsed Time after converting output data to json: 1.33514404296875e-05 seconds
Elapsed Time after sqs data postprocessing: 3.0517578125e-05 seconds
An error occurred (AccessDenied) when calling the SendMessage operation: User: arn:aws:sts::383299343633:assumed-role/AmazonSageMaker-ExecutionRole-20240213T173792/SageMaker is not authorized to perform: sqs:sendmessage on resource: arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs because no identity-based policy allows the sqs:sendmessage action
Elapsed Time after sending data to SQS: 0.07446050643920898 seconds


{'statusCode': 200, 'body': 'Data Sent to SQS Sucessfully'}